In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [28]:
data = pd.read_csv('rfm_xmas19.txt', parse_dates=['trans_date'])
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


In [29]:
data.isnull().sum()

customer_id    0
trans_date     0
tran_amount    0
dtype: int64

In [30]:
data.head()

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78


In [35]:
group_by_customer = data.groupby('customer_id')
last_transaction = group_by_customer['trans_date'].max()
last_transaction


customer_id
FM1112   2019-10-14
FM1113   2019-11-09
FM1114   2019-11-12
FM1115   2019-12-05
FM1116   2019-05-25
            ...    
FM8996   2019-09-09
FM8997   2019-03-28
FM8998   2019-09-22
FM8999   2019-04-02
FM9000   2019-11-28
Name: trans_date, Length: 6889, dtype: datetime64[ns]

In [32]:
cutoff_day =dt.datetime(2019, 10,16)
best_churn = pd.DataFrame(last_transaction)
best_churn['churned']= best_churn['trans_date'].apply(lambda x: 1 if x < cutoff_day else 0)
best_churn.head()

,trans_date,churned
customer_id,,
FM1112,2019-10-14,1
FM1113,2019-11-09,0
FM1114,2019-11-12,0
FM1115,2019-12-05,0
FM1116,2019-05-25,1


Finding a ranking mechanism that can determine a threshold to identify the best customers.


In [39]:
#weighted sum model
best_churn['n_of_transaction']=group_by_customer.size() 
best_churn['amount_spent']= group_by_customer.sum()
best_churn.head()

,churned,n_of_transaction,amount_spent
customer_id,,,
FM1112,1,15,1012
FM1113,0,20,1490
FM1114,0,19,1432
FM1115,0,22,1659
FM1116,1,13,857


In [41]:
best_churn[['n_of_transaction', 'amount_spent']].describe().loc[['min', 'max']]

,n_of_transaction,amount_spent
min,4.0,149.0
max,39.0,2933.0


In [73]:

best_churn['scaled_tran']=(best_churn['n_of_transaction']-
                           best_churn['n_of_transaction'].min())/(best_churn['n_of_transaction'].max()-best_churn['n_of_transaction'].min())

best_churn['scaled_amount'] =(best_churn['amount_spent']-
                              best_churn['amount_spent'].min())/(best_churn['amount_spent'].max()-best_churn['amount_spent'].min())

                          
                         


In [78]:
best_churn["score"] = 100*(.5*best_churn["scaled_tran"] \
                           + .5*best_churn["scaled_amount"])


In [81]:
best_churn.sort_values('score', inplace=True, ascending=False)
best_churn.head()

,churned,n_of_transaction,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM4424,0,39,2933,1.000000,1.000000,100.000000
FM4320,1,38,2647,0.971429,0.897270,93.434934
FM3799,1,36,2513,0.914286,0.849138,88.171182
FM5109,0,35,2506,0.885714,0.846624,86.616892
FM3805,1,35,2453,0.885714,0.827586,85.665025


In [87]:
mean_transaction = data['tran_amount'].mean()
coupon = mean_transaction *0.3
no_customers = 1000/coupon
print( coupon, no_customers, sep='\n')

19.4975736
51.28843314123969


In [89]:
top_50_churned = best_churn[best_churn['churned']==1].head(50)
top_50_churned.to_csv('best_customers.csv')